In [1]:
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
def read_data(drunkfile, normalfile):
    drunk = pd.read_csv(drunkfile)
    normal = pd.read_csv(normalfile)
    data = pd.concat([drunk, normal])
    ans_list = [1]*len(drunk) + [0]*len(normal)
    return data[['smile','anger','contempt','disgust','fear','happiness','neutral','sadness','surprise','eyeOccluded']].values, ans_list

In [4]:
X_train, Y_train = read_data("drunk.csv", "normal.csv")

In [5]:
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [7]:
X_train[X_train == True] = 1
X_train[X_train == False] = 0

In [8]:
(X_train, Y_train) = shuffle(X_train, Y_train)

In [9]:
X_train, X_valid = X_train[:-50], X_train[-50:]
Y_train, Y_valid = Y_train[:-50], Y_train[-50:]

In [10]:
model = Sequential()
model.add(Dense(input_dim = X_train.shape[1], units = 10, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 8, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
callbacks = []
callbacks.append(ModelCheckpoint('models/model-{epoch:05d}-{val_acc:.5f}-{val_loss:.5f}.h5', monitor='val_acc', save_best_only=True, mode = 'auto', period=1))
model.fit(X_train, Y_train, batch_size = 8, epochs = 100, validation_split=0.2, callbacks=callbacks)

Train on 1018 samples, validate on 255 samples
Epoch 1/100
1008/1018 [============================>.] - ETA: 0s - loss: 0.6487 - acc: 0.6260  

ImportError: `save_model` requires h5py.

In [ ]:
result = model.evaluate(X_valid, Y_valid, batch_size = X_valid.shape[0])
print(result[1])

In [ ]:
from xgboost import XGBClassifier
model2 = XGBClassifier(max_depth=3, n_estimators=200, learning_rate=0.05)
model2.fit(X_train, Y_train.flatten())
predict_train = model2.predict(X_train)
acc_train = np.mean(Y_train.flatten() == predict_train)
print("acc_train = ", acc_train)
predict_valid = model2.predict(X_valid)
acc_valid = np.mean(Y_valid.flatten() == predict_valid)
print("acc_valid = ", acc_valid)

In [ ]:
import pickle
pickle.dump(model2, open("xgbmodel", "wb"))

In [ ]:
loaded_model = pickle.load(open("xgbmodel", "rb"))
acc_valid = np.mean(Y_valid.flatten() == predict_valid)
print("acc_valid = ", acc_valid)

In [9]:
from keras.models import load_model
model = load_model("keras_model.h5")
result = model.evaluate(X_valid, Y_valid, batch_size = X_valid.shape[0])
print(result[1])

ImportError: `load_model` requires h5py.

In [ ]:
temp = X_valid[0].reshape((1,10))
ans = model.predict(X_valid[0].reshape((1,10)))
print(ans)